Problema 3:

En este problema trabajaremos con el archivo llamado reactiva.xlsx
Deberá realizar lo siguiente:
- Cree un archivo llamado procesamiento .py
- Genere una función de limpieza que permita el renombre de las columnas eliminando espacios, tildes y convirtiendo los nombres de columna a minúscula. De ser necesario cambie el nombre de columna a uno que le sea de más ayuda.
- Elimine la columna ID y TipoMoneda que se encuentre repetida.
- De la columna “DISPOSITIVO LEGAL” elimine el carácter coma ‘,’ del texto.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html
Ejemplo
df[columnas] = df[columnas].replace({'0m':''}, regex=True)
- Empleando el api de sunat de la pc4. Dolarice los valores de montos de inversión y montos de transferencia de acuerdo con el valor actual del dólar. Deberá mostrar estos montos en dos nuevas columnas.
- Para la columna “Estado” cambie los valores a: Actos Previos, Concluido, Resuelto y Ejecucion
- Cree una nueva columna que puntue el estado según: ActosPrevios -> 1, Resuelto->0, Ejecución 2 y Concluido 3
Genere los siguientes reportes
- Almacene en una base de datos una tabla de ubigeos a partir de los datos sin duplicados de las columnas “ubigeo”, “Region”, “Provincia”, “Distrito”
- Por cada región genere un Excel con el top 5 costo inversión de las obras de tipo Urbano en estado 1,2,3. En caso no haber datos en alguna de las regiones, no se generará el reporte
Generación envio de correo:
- Cree un modulo envio_correo.py que se encargue de la parte de envio de correo. Puede emplear funciones y/o clases
Integrar los módulos y tomar captura al menos de uno de los correos enviados mediante código.

In [12]:
import pandas as pd
reactiva = pd.read_excel("./reactiva.xlsx",header = 1)

In [13]:
reactiva.dtypes

ITEM                             int64
UBIGEO                          object
SNIP                             int64
CUI                              int64
REGION                          object
PROVINCIA                       object
DISTRITO                        object
PROYECTO                        object
NORMA                           object
DISPOSITIVO_2                   object
AMBITO                          object
UNIDAD EJECUTORA                object
TOTAL EMPLEOS                    int64
POBLACION BENEFICIARIA           int64
TIPOLOGIA                       object
MONTO DE INVERSIÓN             float64
MONTO DE TRANSFERENCIA 2020      int64
PLAZO DE EJECUCIÓN               int64
TIPO DE PROCESO                 object
ESTADO  SSP                     object
SUB ESTADO SSP                  object
dtype: object

In [ ]:
def limpiar_y_renombrar_columnas(df):
    # Renombrar las columnas eliminando espacios, tildes y convirtiendo a minúsculas
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
    # Eliminar columnas ID y TipoMoneda duplicadas
    df = df.loc[:,~df.columns.duplicated()]
    
    # Eliminar comas de la columna DISPOSITIVO_LEGAL
    df['dispositivo_legal'] = df['dispositivo_legal'].str.replace(',', '')
    
    return df


In [ ]:
def eliminar_columnas_duplicadas_y_comas(df):
    # Eliminar columnas ID y TipoMoneda duplicadas
    df = df.loc[:,~df.columns.duplicated()]
    
    # Eliminar comas de la columna DISPOSITIVO_LEGAL
    df['dispositivo_legal'] = df['dispositivo_legal'].str.replace(',', '')

    return df

In [ ]:
import requests

def obtener_tipo_cambio_dolar_sunat():
    try:
        response = requests.get('https://api.apis.net.pe/v1/tipo-cambio-sunat')
        data = response.json()
        tipo_cambio = data['data']['tipocambio'][0]['venta']
        return tipo_cambio
    except Exception as e:
        print(f"Error al obtener el tipo de cambio del dólar: {e}")
        return None

In [ ]:
def cambiar_valores_y_puntuar_estado(df):
    # Cambiar los valores de la columna "Estado"
    df['estado'] = df['estado'].replace({
        'ActosPrevios': 'Actos Previos',
        'Concluído': 'Concluido'
    })

    # Crear una nueva columna que puntúe el estado
    df['estado_puntuado'] = df['estado'].map({
        'Actos Previos': 1,
        'Resuelto': 0,
        'Ejecución': 2,
        'Concluido': 3
    })

    return df

In [ ]:
import sqlite3

def generar_tabla_ubigeos(df):
    # Seleccionar las columnas relevantes y eliminar duplicados
    ubigeos = df[['UBIGEO', 'REGION', 'PROVINCIA', 'DISTRITO']].drop_duplicates()

    # Conectar a la base de datos SQLite
    conn = sqlite3.connect('ubigeos.db')

    # Guardar la tabla de ubigeos en la base de datos
    ubigeos.to_sql('ubigeos', conn, if_exists='replace', index=False)

    # Cerrar la conexión a la base de datos
    conn.close()

    print("Tabla de ubigeos generada y almacenada en la base de datos 'ubigeos.db'")

In [ ]:
def generar_reporte_top5_por_region(df):
    # Filtrar las obras de tipo "Urbano"
    obras_urbanas = df[df['TIPOLOGIA'] == 'Urbano']

    # Crear un diccionario para almacenar los DataFrames de las regiones
    regiones_dataframes = {}

    # Iterar sobre cada región
    for region in obras_urbanas['REGION'].unique():
        # Filtrar las obras de la región actual
        obras_region = obras_urbanas[obras_urbanas['REGION'] == region]

        # Filtrar las obras en estado 1, 2 y 3
        obras_estado1 = obras_region[obras_region['estado_puntuado'] == 1].nlargest(5, 'MONTO DE INVERSIÓN')
        obras_estado2 = obras_region[obras_region['estado_puntuado'] == 2].nlargest(5, 'MONTO DE INVERSIÓN')
        obras_estado3 = obras_region[obras_region['estado_puntuado'] == 3].nlargest(5, 'MONTO DE INVERSIÓN')

        # Agregar los DataFrames al diccionario
        regiones_dataframes[region] = {
            'Estado 1': obras_estado1,
            'Estado 2': obras_estado2,
            'Estado 3': obras_estado3
        }

    # Generar un archivo Excel para cada región
    for region, dataframes in regiones_dataframes.items():
        writer = pd.ExcelWriter(f'Top5_Obras_{region}.xlsx')
        for estado, df_estado in dataframes.items():
            df_estado.to_excel(writer, sheet_name=estado, index=False)
        writer.save()

    print("Reportes generados correctamente.")

# Llamar a la función para generar los reportes
generar_reporte_top5_por_region(df)


In [ ]:
import pandas as pd
import sqlite3

def generar_tabla_ubigeos(df):
    # Seleccionar las columnas relevantes y eliminar duplicados
    ubigeos = df[['UBIGEO', 'REGION', 'PROVINCIA', 'DISTRITO']].drop_duplicates()

    # Conectar a la base de datos SQLite
    conn = sqlite3.connect('ubigeos.db')

    # Guardar la tabla de ubigeos en la base de datos
    ubigeos.to_sql('ubigeos', conn, if_exists='replace', index=False)

    # Cerrar la conexión a la base de datos
    conn.close()

    print("Tabla de ubigeos generada y almacenada en la base de datos 'ubigeos.db'")

def generar_reporte_top5_por_region(df):
    # Filtrar las obras de tipo "Urbano"
    obras_urbanas = df[df['TIPOLOGIA'] == 'Urbano']

    # Crear un diccionario para almacenar los DataFrames de las regiones
    regiones_dataframes = {}

    # Iterar sobre cada región
    for region in obras_urbanas['REGION'].unique():
        # Filtrar las obras de la región actual
        obras_region = obras_urbanas[obras_urbanas['REGION'] == region]

        # Filtrar las obras en estado 1, 2 y 3
        obras_estado1 = obras_region[obras_region['estado_puntuado'] == 1].nlargest(5, 'MONTO DE INVERSIÓN')
        obras_estado2 = obras_region[obras_region['estado_puntuado'] == 2].nlargest(5, 'MONTO DE INVERSIÓN')
        obras_estado3 = obras_region[obras_region['estado_puntuado'] == 3].nlargest(5, 'MONTO DE INVERSIÓN')

        # Agregar los DataFrames al diccionario
        regiones_dataframes[region] = {
            'Estado 1': obras_estado1,
            'Estado 2': obras_estado2,
            'Estado 3': obras_estado3
        }

    # Generar un archivo Excel para cada región
    for region, dataframes in regiones_dataframes.items():
        writer = pd.ExcelWriter(f'Top5_Obras_{region}.xlsx')
        for estado, df_estado in dataframes.items():
            df_estado.to_excel(writer, sheet_name=estado, index=False)
        writer.save()

    print("Reportes generados correctamente.")

# Ejemplo de uso
# Primero, limpiar y preparar los datos
# df = limpiar_y_renombrar_columnas(df)
# df = eliminar_columnas_duplicadas_y_comas(df)
# df = cambiar_valores_y_puntuar_estado(df)

# Luego, generar la tabla de ubigeos
# generar_tabla_ubigeos(df)

# Finalmente, generar los reportes por región
# generar_reporte_top5_por_region(df)


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def enviar_correo(destinatario, asunto, mensaje):
    # Configuración del servidor SMTP
    servidor_smtp = 'smtp.example.com'
    puerto_smtp = 587
    usuario_smtp = 'tu_correo@example.com'
    contraseña_smtp = 'tu_contraseña'

    # Crear objeto MIME multipart
    correo = MIMEMultipart()
    correo['From'] = usuario_smtp
    correo['To'] = destinatario
    correo['Subject'] = asunto

    # Agregar cuerpo del mensaje
    correo.attach(MIMEText(mensaje, 'plain'))

    # Conectar y autenticar con el servidor SMTP
    with smtplib.SMTP(servidor_smtp, puerto_smtp) as servidor:
        servidor.starttls()
        servidor.login(usuario_smtp, contraseña_smtp)
        # Enviar correo
        servidor.send_message(correo)

    print("Correo enviado con éxito a", destinatario)